In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
############################################
################# Load data ################
############################################

#
def get_sessions_from_directory_listing(root_dir,
                                       animal_id):
    
    sessions = os.listdir(root_dir+animal_id)
    print ("sessions: ", sessions)
    
    return sessions
    
#
animal_ids = [
    
    'DON-002865',
    'DON-003165',
    'DON-003343',
            
    'DON-006084',
    'DON-006085',
    'DON-006087',
    

#     'DON-008497',
#     'DON-008498',
#     'DON-008499',
    
    # 'DON-009191',
    # 'DON-009192',
    # 'DON-010473',
    # 'DON-010477',
    

       
    
    
]
              
############################################
############################################
############################################
root_dir = '/media/cat/4TB/donato/'
dir_ = '/002P-F/tif/'

#
for animal_id in animal_ids:

    #
    print ("Processing: ", animal_id)

    #
    sessions = get_sessions_from_directory_listing(root_dir,
                                                  animal_id)

    #
    for session in sessions:
        #
        fname_check  =  os.path.join(root_dir, 
                                     animal_id, 
                                     session+dir_, 'suite2p','plane0',
                                     'goodcell_correlations_array_post_deduplication_filtered.npy'
                                      )

        #
        #if os.path.exists(fname_check):
        #   continue

        #
        try: 
        #if True:
            #
            print ("processing: ", session)

            #if session != '20220302':
            #    continue

            c = calcium.Calcium()
            c.root_dir = root_dir
            c.data_dir = os.path.join(root_dir, animal_id, session+dir_, 'suite2p','plane0')
            c.animal_id = animal_id
            c.session = session
            c.detrend_model_order = 1
            
            #
            c.load_suite2p()           #note: this already deletes the non-trusted suite2p cells
            c.save_python = True
            c.save_matlab = False

            # 
            c.load_binarization()
            binarization_method = 'upphase'
            if binarization_method=='onphase':
                traces = c.F_onphase_bin
            elif binarization_method=='upphase':
                traces = c.F_upphase_bin
            else:
                print ("METHOD NOT FOUND")

            #print ("binarized data: ", traces.shape)

            ###################################################################
            ########## cleanup cells + compute pairwise correlations ##########
            ###################################################################
            c.load_footprints()
            c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
            c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
            c.corr_max_percent_overlap = 0.25       # max overlap for overlap method
            c.corr_threshold = 0.3                  # max correlation allowed for high overlap

            #  
            c.corr_delete_method = 'lowest_snr' #'highest_connected', lowest_snr'
            c.recompute_deduplication = True

            # compute both here
            corr_types = ['filtered', 'upphase']
            c.self_ncores = 16
            for corr_type in corr_types:
                c.correlation_datatype = corr_type      # filtered vs. upphase

                # 
                c.recompute_correlation = True
                c.compute_correlations()

                #
                c.remove_duplicate_neurons()     # this removes duplicate neurons and saves non-duplicate version of correlation array

                        
        except:
           print ("... errror loading : ", session)

print ("DONE...")

Processing:  DON-002865
sessions:  ['DON-002865_20210210', 'DON-002865_202102114', 'DON-002865_202102115', 'DON-002865_202102113', 'DON-002865_202102119', 'DON-002865_20210211', 'DON-002865_202102116', 'DON-002865_202102117', 'DON-002865_202102118', 'DON-002865_202102112']
processing:  DON-002865_20210210
... computing pairwise pearson correlation ...
 RASTERS IN:  (1064, 86186)


100%|██████████| 100/100 [00:24<00:00,  4.10it/s]


Parallel corrs:  (1132096, 4)


100%|██████████| 1132096/1132096 [00:00<00:00, 1748673.41it/s]

... computing cell overlaps ...


...

Parallel corrs:  (583696, 4)


100%|██████████| 583696/583696 [00:00<00:00, 1755043.89it/s]

... computing cell overlaps ...



100%|██████████| 30/30 [00:52<00:00,  1.73s/it]

pre clean corr array:  (764, 764, 2)
clean cell ids:  (607,)
pre clean corr array:  (607, 607)
DONE...
